In [1]:
import numpy as np
import cv2

In [31]:
img = cv2.imread('/home/optolab/ur2020_workspace/src/telemove/src/saved/img_1.png')
#img = cv2.flip(img, 0)
#img = cv2.flip(img, 1)

In [3]:
'''img (CvArr) – Image where the circle is drawn
center (CvPoint) – Center of the circle
radius (int) – Radius of the circle
color (CvScalar) – Circle color
thickness (int) – Thickness of the circle outline if positive, otherwise this indicates that a filled circle is to be drawn
lineType (int) – Type of the circle boundary, see Line description
shift (int) – Number of fractional bits in the center coordinates and radius value'''

cv2.circle(img, (112,107), 5, (0,0,255), thickness=-1, lineType=8, shift=0)
cv2.circle(img, (137,107), 5, (0,0,255), thickness=-1, lineType=8, shift=0)
cv2.circle(img, (113,105), 5, (0,255,0), thickness=-1, lineType=8, shift=0)
# (112,107)
# (137,107)
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
# intrinsic matrix

K = np.array([[1.06314129e+03, 0.00000000e+00, 9.62163539e+02],
              [0.00000000e+00, 1.06450150e+03, 5.38537956e+02],
              [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
D = np.array([[ 0.04147518, -0.0393437,   0.00013869, -0.00071462, -0.00889844]])
R = np.array([[ 0.99965889, -0.01399777, -0.02204921],
              [ 0.01331178,  0.99943206, -0.03095737],
              [ 0.02247002,  0.03065329,  0.99927748]])
t = np.array([[-33.35386641], [-16.99836057], [ 41.93586922]])
Rt = np.array([[0.99965889, -0.01399777, -0.02204921, -33.35386641],
              [0.01331178,  0.99943206, -0.03095737, -16.99836057],
              [0.02247002,  0.03065329,  0.99927748, 41.93586922]])

reference, _ = cv2.projectPoints(np.array([[0.0, 0.0, 0.0]]), R, t, K, D)
reference = reference.flatten()
reference = np.array([[reference[0], reference[1], 1.0]])




'''K = np.array([[1058.70943117763, 0.0, 966.890434497565],
             [0.0, 1061.39848586722, 537.457947049765],
             [0.0, 0.0, 1.0]])
# rotation matrix R
R = np.array([[0.999627752961109, -0.0212705783300841, -0.0170856081960356],
              [0.0199628781001384, 0.997105046192065, -0.0733690013307973],
              [0.0185967472393344, 0.0730006120236213, 0.997158498753479]])
# translation matrix t
t = np.array([[-311.795183459492, -367.888491366132, 986.881541982339]])
t = t.T

Rt = np.array([[0.999627752961109, -0.0212705783300841, -0.0170856081960356, -311.795183459492],
              [0.0199628781001384, 0.997105046192065, -0.0733690013307973, -367.888491366132],
              [0.0185967472393344, 0.0730006120236213, 0.997158498753479, 986.881541982339]])

Rt2 = np.array([[0.999627752961109, -0.0212705783300841, -0.0170856081960356],
               [0.0199628781001384, 0.997105046192065, -0.0733690013307973],
               [0.0185967472393344, 0.0730006120236213, 0.997158498753479],
               [-311.795183459492, -367.888491366132, 986.881541982339]])'''

#pt = np.array([[1315.0, 283.0, 1.0]])
#pt2 = np.array([[1215.0 ,283.0, 1.0]])
pt = np.array([[112.0, 131.0, 1.0]])
pt2 = np.array([[137.0 ,107.0, 1.0]])
pt3 = np.array([[0.0, 0.0, 0.0]]) # corrisponde a pt4 trovato con project
pt4 = np.array([[112.67342515, 105.18766658, 1.0]])

In [292]:
pt = pt.T
pt2 = pt2.T
K2 = np.linalg.inv(K)
R2 = np.linalg.inv(R)

In [293]:
S = K2.dot(pt)
N = S - t
PT = R2.dot(N)

In [294]:
S = K2.dot(pt2)
N = S - t
PT2 = R2.dot(N)

In [295]:
S = K2.dot(pt4.T)
N = S - t
PT3 = R2.dot(N)

In [297]:
print(PT)
print(PT2)
print(PT3)

#PT2 - PT
print((PT - PT3)*100)
print((PT2 - PT3)*100)

[[ 31.8444427 ]
 [ 14.89557434]
 [-42.13845872]]
[[ 31.86764978]
 [ 14.87271222]
 [-42.13827926]]
[[ 31.84475313]
 [ 14.87133096]
 [-42.13772203]]
[[-0.03104258]
 [ 2.42433777]
 [-0.07366963]]
[[ 2.28966475]
 [ 0.13812574]
 [-0.05572308]]


In [251]:
C = pt3.dot(Rt)
C3 = K.dot(C)
print(C3)

ValueError: shapes (3,4) and (3,1) not aligned: 4 (dim 1) != 3 (dim 0)

In [301]:
imgpts, jac = cv2.projectPoints(pt3, R, t, K, D)

In [5]:
########qui
def px2meters(pt, reference, K, R, t):
    ''' Funzione di conversione tra pixel e metri per ottenere la posizione
    corretta in metri della posizione rilevata nell'immagine RGB-D. La Kinect
    prende (x,y) dall'RGB e (z) dal sensore di profondita', quindi deve essere
    calibrato per forza. Calibrazione fatta in MATLAB! Riporto qui le matrici '''

    # pt e' px
    
    K2 = np.linalg.inv(K)
    R2 = np.linalg.inv(R)
    pt = pt.T

    S = K2.dot(pt)
    print('S = K-1 * pt: \n' + str(S))
    N = S - t
    print('N = S - t: \n' + str(N))
    PT = R2.dot(N)
    print('PT = R-1 * N: \n' + str(PT)) 
    print(reference)
    XYZ = PT # in cm

    return XYZ

In [18]:
ptr = np.array([[69.0, 243.0, 1.0]]) #pixel
#513.0, 241.0, 1.0

In [38]:
px2meters(ptr, reference, K, R, t)

[[ 69.]
 [243.]
 [  1.]]
[[-0.84011744]
 [-0.27763038]
 [ 1.        ]]
[[ 33.2503098 ]
 [ 18.60576244]
 [-40.79492695]]
[[ 31.93997964]
 [ 16.57912217]
 [-42.67263688]]
[[93.03087507 56.98615768  1.        ]]


array([[-61.09089544],
       [-40.40703551],
       [-43.67263688]])

In [43]:
pte, _ = cv2.projectPoints(np.array([[32.91246167, 17.37687301, -42.8673002]]), R, t, K, D)
print(pte)

[[[ 76.9499687  154.63142064]]]


In [15]:
K = np.array([[1.06314129e+03, 0.00000000e+00, 9.62163539e+02],
              [0.00000000e+00, 1.06450150e+03, 5.38537956e+02],
              [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
Kopt = np.array([[1.04808667e+03, 0.00000000e+00, 9.59995485e+02],
                 [0.00000000e+00, 1.04803015e+03, 5.38729787e+02],
                 [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
D = np.array([[ 0.04147518, -0.0393437, 0.00013869, -0.00071462, -0.00889844]])
Dnull = np.array([[ 0.0, 0.0, 0.0, 0.0, 0.0]])



RD = np.array([[0.99916424, -0.01386533, -0.03845227],
             [0.01251816,  0.99930691, -0.03505704],
             [0.0389117,   0.03454639,  0.9986453]])
tD = np.array([[-35.12119381], [-19.63899217], [41.9282398]])


R = np.array([[ 0.99918976, -0.01526008, -0.03724185],
              [ 0.01386541,  0.99920327, -0.03742418],
              [ 0.03778327,  0.03687748,  0.99860527]])
t = np.array([[-34.09042724], [-18.88339282], [ 41.79492695]])

    
reference, _ = cv2.projectPoints(np.array([[0.0, 0.0, 0.0]]), R, t, K, D)
reference = reference.flatten()
print('Reference system in pixel coordinates: ' +str(reference))
reference, _ = cv2.projectPoints(np.array([[0.0, 0.0, 0.0]]), R, t, K, Dnull)
reference = reference.flatten()
print('Reference system NOT DISTORTED in pixel coordinates: ' +str(reference))
reference = np.array([[reference[0], reference[1], 1.0]]) #px
newref = np.array([[79.90819, 47.1515, 1.0]]) 
print('Difference between projected and refined: ' + str(newref - reference))

ptmeasure, _ = cv2.projectPoints(np.array([[0.0, 8.0, 0.0]]), R, t, K, D)
ptmeasure = ptmeasure.flatten()
ptmeasure = np.array([[ptmeasure[0], ptmeasure[1], 1.0]]) #px
print('To measure: ' + str(ptmeasure))
print('measured point: ' + str(ptmeasure - reference))

Reference system in pixel coordinates: [93.03087507 56.98615768]
Reference system NOT DISTORTED in pixel coordinates: [95.0022963 57.5848488]
Difference between projected and refined: [[-15.0941063 -10.4333488   0.       ]]
To measure: [[ 91.38663813 261.30057004   1.        ]]
measured point: [[ -3.61565817 203.71572124   0.        ]]


In [13]:
pt = np.array([[93.0, 187.0, 1.0]]) #image point
ref = np.array([[93.0, 57.0, 1.0]])

def prova(pt, K, R, t):
    R2 = np.linalg.inv(R)
    K2 = np.linalg.inv(K)
    temp = R2.dot(K2) #3x3
    temp = temp.dot(pt.T) #3x3 x 3x1

    temp2 = R2.dot(t)
    s = 0 + temp2[-1]
    s = s/temp[-1]
    #print(s)

    world = s*K2
    #world = K2
    world = world.dot(pt.T) - t
    world = R2.dot(world)
    print(world)
    return world
    
uno = prova(pt,K,R,t)
due = prova(ref,K,R,t)
print((uno-due))

#SBAGLIATA

[[-1.49090069e-01]
 [ 5.02785717e+00]
 [ 5.49560397e-15]]
[[-7.61238739e-02]
 [-2.02838087e-02]
 [ 1.78676518e-16]]
[[-7.29661952e-02]
 [ 5.04814098e+00]
 [ 5.31692745e-15]]


In [9]:
pt = np.array([[93.0, 57.0, 1.0]])
ref = px2meters(pt, reference, K, R, t)
pt = np.array([[93.0, 187.0, 1.0]])
xyz = px2meters(pt, reference, K, R, t)
print('Difference: ' + str((xyz-ref)*100))

S = K-1 * pt: 
[[-0.81754283]
 [-0.45236005]
 [ 1.        ]]
N = S - t: 
[[ 33.27288441]
 [ 18.43103277]
 [-40.79492695]]
PT = R-1 * N: 
[[ 31.96011326]
 [ 16.40418721]
 [-42.66693848]]
[[93.03087507 56.98615768  1.        ]]
S = K-1 * pt: 
[[-0.81754283]
 [-0.33023716]
 [ 1.        ]]
N = S - t: 
[[ 33.27288441]
 [ 18.55315566]
 [-40.79492695]]
PT = R-1 * N: 
[[ 31.96180654]
 [ 16.52621281]
 [-42.67150883]]
[[93.03087507 56.98615768  1.        ]]
Difference: [[ 0.1693284 ]
 [12.20255922]
 [-0.45703485]]


In [17]:
img = cv2.imread('/home/optolab/ur2020_workspace/src/telemove/src/saved/img_1.png') #antidistorta
#img = cv2.flip(img, 0)
#img = img[230:830, 540:1450]
cv2.circle(img, (93, 57), 5, (0,255,0), thickness=-1, lineType=8, shift=0)
cv2.circle(img, (93, 187), 5, (255,0,0), thickness=-1, lineType=8, shift=0)
cv2.circle(img, (95, 58), 5, (255,0,0), thickness=-1, lineType=8, shift=0)

cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [44]:
pt = np.array([[22.5, 0.0]])
#b1 = 0.38, -0.04

In [45]:
RW = np.array([[-0.0001, 0.0100], [-0.0100, -0.0001]])
tW = np.array([[-0.0457],[0.3835]])
Pr = RW.dot(pt.T)
Pr = Pr + tW

In [46]:
Pr

array([[-0.04795],
       [ 0.1585 ]])